In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression

# Read data
df = pd.read_csv('train.csv', index_col='Id')

df_encoded = pd.get_dummies(df)

X = df_encoded.drop('Verkaufspreis', axis=1)
y = df_encoded['Verkaufspreis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Deleting missing values, since there are only less than 1% of missing values
df = df.dropna(subset=['Wohngebiet', 'Funktionalitaet', 'KuechenQualitaet', 'Elektrik',
                       'Verkaufstyp', 'Garagenflaeche', 'Garagenautos'])

# Fill missing values with Linear Regression prediction in feature 'Strassenlaenge'
model = LinearRegression()
model.fit(np.array(df[df['Strassenlaenge'].notna()].Verkaufspreis).reshape(-1, 1), 
          df[df['Strassenlaenge'].notna()].Strassenlaenge)
predicted_values = model.predict(np.array(df[df['Strassenlaenge'].isna()].Verkaufspreis).reshape(-1, 1))
df.loc[df['Strassenlaenge'].isnull(), 'Strassenlaenge'] = predicted_values


# Fill missing values with mode
features_fillna_mode = ['Funktionalitaet', 'KuechenQualitaet', 'Elektrik', 'Garagenbaujahr',
                        'Garageninnenausbau', 'Garagentyp','Versorgung', 'Kellerbereichgroesse2', 'GeringequalitaetFlaeche',
              'Zufahrtsweg', 'Sondermerkmal', 'Poolqualitaet',
              'Garagenqualitaet', 'Garagenzustand']
for feature in features_fillna_mode:
    df[feature] = df[feature].fillna(df[feature].mode()[0])


# Fill missing values with Nan
features_fillna_nan = ['Kellerhoehe', 'Kellerzustand', 'Kellerbelichtung', 
                     'Kellerbereich1', 'Kellerbereich2', 'Kaminqualitaet',
                     'Zaunqualitaet']
for feature in features_fillna_nan:
    df[feature] = df[feature].fillna('Nan')


# Fill missing values with 0.0
features_fillna_0 = ['Kellerbereichgroesse1', 'Kellerbereichgroesse2', 'KellerbereichgroesseGes', 
                     'KellerbereichgroesseNAu', 'KellerVollbadezimmer', 'KellerHalbbadezimmer',
                     'Mauerwerkflaeche']
for feature in features_fillna_0:
    df[feature] = df[feature].fillna(0.0)


# Fill Mauerwerktyp with 'Kein'
df['Mauerwerktyp'] = df['Mauerwerktyp'].fillna('Kein')


In [13]:
# Limit 'Grundstuecksgroesse' with 6000 m2
df = df[df['Grundstuecksgroesse'] < 6000]

# Limit 'Strassenlaenge' with 60 m
df = df[df['Strassenlaenge'] <= 60]

# Delete instance 714 because there Baujahr is bigger than Umbaujahr
df = df[df['Umbaujahr'] >= df['Baujahr']]

# Delete Garagenbaujahr bigger than 2023
df = df[df['Garagenbaujahr'] < 2023]

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
# Create a random forest regressor
forest = RandomForestRegressor(random_state=42, n_estimators=800,min_samples_split=2,min_samples_leaf=1,max_features='sqrt',max_depth=40,bootstrap=False )

# Train the model
forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)

r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [38]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [44]:

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 300, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)
print(rf_random.best_params_)

Fitting 3 folds for each of 300 candidates, totalling 900 fits
{'n_estimators': 800, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 40, 'bootstrap': False}


In [45]:
y_pred = rf_random.best_estimator_.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

R^2 Score: 0.9184274607657424
Mean Squared Error: 292670280.46184844


In [47]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True, False],
    'max_depth': [40,60,80,100],
    'max_features': [2, 3],
    'min_samples_leaf': [1,2,3,4],
    'min_samples_split': [2, 4,6,8],
    'n_estimators': [600, 700, 800, 900, 1000]
}
rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(X_train,y_train)
print(grid_search.best_params_)


Fitting 3 folds for each of 1280 candidates, totalling 3840 fits
{'bootstrap': False, 'max_depth': 40, 'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000}


In [48]:
y_pred = grid_search.best_estimator_.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

R^2 Score: 0.8936510916785985
Mean Squared Error: 381564250.87922907


In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test)
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

R^2 Score: 0.8936510916785985
Mean Squared Error: 381564250.87922907
